In [1]:
import qiskit
from qiskit import *
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, StatevectorSimulator
from qiskit_aer.quantum_info import AerStatevector, AerDensityMatrix
from qiskit_ibm_runtime import Session
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCMT, CZGate, GroverOperator

In [2]:
def encode_winner(circuit, num, bits):
  for i in range(bits):
    if not num & (1 << i):
      circuit.x(i)

  circuit.append(MCMT(CZGate(), bits-1, 1), range(bits))

  for i in range(bits):
    if not num & (1 << i):
      circuit.x(i)


def make_oracle(list_n, k, bits=5):
  oracle = QuantumCircuit(bits)
  oracle.h(range(bits))

  for i in range(len(list_n)):
    if list_n[i] < k:
      encode_winner(oracle, i, bits)

  return oracle

def make_grover(oracle, bits=5):
  grover = GroverOperator(oracle)
  grover.measure_all()
  return grover

def run_circuit(circuit):
  sim = StatevectorSimulator()
  circuit = qiskit.transpile(circuit, sim)
  counts = sim.run(circuit, shots=1000).result().get_counts()
  return counts


In [3]:
NUM_QBITS=5

def less_than_k(k, list_n):
  oracle = make_oracle(list_n, k, NUM_QBITS)
  circuit = make_grover(oracle)
  result = run_circuit(circuit).int_outcomes()
  solution = [list_n[x] for x in result if result[x] > 100]
  return solution

list_n = [4,9,11,14,1,13,6,15]
k = 7
print(less_than_k(k, list_n))

[6, 4, 1]


## **Circuit Diagrams**

In [4]:
oracle = make_oracle(list_n, k, NUM_QBITS)
oracle.draw()

┌───┐┌───┐┌───────┐┌───┐┌───┐┌───────┐┌───┐┌───┐┌───────┐┌───┐
q_0: ┤ H ├┤ X ├┤0      ├┤ X ├┤ X ├┤0      ├┤ X ├┤ X ├┤0      ├┤ X ├
     ├───┤├───┤│       │├───┤├───┤│       │├───┤└───┘│       │└───┘
q_1: ┤ H ├┤ X ├┤1      ├┤ X ├┤ X ├┤1      ├┤ X ├─────┤1      ├─────
     ├───┤├───┤│       │├───┤└───┘│       │└───┘     │       │     
q_2: ┤ H ├┤ X ├┤2 mcmt ├┤ X ├─────┤2 mcmt ├──────────┤2 mcmt ├─────
     ├───┤├───┤│       │├───┤┌───┐│       │┌───┐┌───┐│       │┌───┐
q_3: ┤ H ├┤ X ├┤3      ├┤ X ├┤ X ├┤3      ├┤ X ├┤ X ├┤3      ├┤ X ├
     ├───┤├───┤│       │├───┤├───┤│       │├───┤├───┤│       │├───┤
q_4: ┤ H ├┤ X ├┤4      ├┤ X ├┤ X ├┤4      ├┤ X ├┤ X ├┤4      ├┤ X ├
     └───┘└───┘└───────┘└───┘└───┘└───────┘└───┘└───┘└───────┘└───┘

In [5]:
circuit = make_grover(oracle)
circuit.draw()

┌────┐ ░ ┌─┐            
state_0: ┤0   ├─░─┤M├────────────
         │    │ ░ └╥┘┌─┐         
state_1: ┤1   ├─░──╫─┤M├─────────
         │    │ ░  ║ └╥┘┌─┐      
state_2: ┤2 Q ├─░──╫──╫─┤M├──────
         │    │ ░  ║  ║ └╥┘┌─┐   
state_3: ┤3   ├─░──╫──╫──╫─┤M├───
         │    │ ░  ║  ║  ║ └╥┘┌─┐
state_4: ┤4   ├─░──╫──╫──╫──╫─┤M├
         └────┘ ░  ║  ║  ║  ║ └╥┘
 meas: 5/══════════╩══╩══╩══╩══╩═
                   0  1  2  3  4